<a href="https://colab.research.google.com/github/pyagoubi/Credit-Default-Prediction/blob/main/Amex_LGBM_fin_pd_V20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall lightgbm

In [ ]:
!pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"

In [1]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#set WD
import os
os.chdir('/content/drive/MyDrive/Amex/parquet')

# XGBoost 


# Load Libraries

In [3]:
# LOAD LIBRARIES
import pandas as pd, numpy as np # CPU libraries
#import cupy, cudf # GPU libraries
import matplotlib.pyplot as plt, gc, os

from sklearn.model_selection import KFold
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import csv, itertools
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import xgboost

from sklearn.model_selection import train_test_split, StratifiedKFold
from xgboost import plot_importance
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import lightgbm as lgb

import itertools


#print('RAPIDS version',cudf.__version__)

In [4]:
# VERSION NAME FOR SAVED MODEL FILES
VER = 20

# TRAIN RANDOM SEED
SEED = 42

# FILL NAN VALUE
NAN_VALUE = -127 # will fit in int8

# FOLDS PER MODEL
FOLDS = 5

NUM_PARTS = 10


TRAIN_PATH = 'train.parquet'
TEST_PATH = 'test.parquet'
TARGET_PATH = 'train_labels.csv'
SAVE_PATH = '/content/drive/MyDrive/Amex/parquet/XGB final/'
SUBMISSION_FILE_PATH = '/content/drive/MyDrive/Amex/parquet/sample_submission.csv'

# Process and Feature Engineer Train Data

In [5]:
def read_file(path = '', usecols = None):
    # LOAD DATAFRAME
    if usecols is not None: df = pd.read_parquet(path, columns=usecols)
    else: df = pd.read_parquet(path) #df = cudf.read_parquet(path)
    df['customer_ID'] = df['customer_ID'].str[-16:].apply(int, base =16)
    df.S_2 = pd.to_datetime( df.S_2 )
    print('shape of data:', df.shape)
    return df

def revertnan(df):
  df[df==-1] = np.nan 
  return df

def fill_na(df, NAN_VALUE):
  df = df.fillna(NAN_VALUE)
  return df

def numberobs_feature(df):
  df['number_of_observations'] = df.groupby('customer_ID')['customer_ID'].transform('count')
  df.loc[df['B_33'].isnull() & (df.number_of_observations==1),'number_of_observations'] = 0.5
  return df

def afterpay(df):
  # compute "after pay" features
  for bcol in [f'B_{i}_last' for i in [11,14,17]]+['D_39_last','D_131_last']+[f'S_{i}_last' for i in [16,23]]:
    for pcol in ['P_2_last','P_3_last']:
      if bcol in df.columns:
        df[f'{bcol}-{pcol}'] = df[bcol] - df[pcol]
  return df

def get_features(df):
  all_cols = [c for c in list(df.columns) if c not in ['customer_ID','S_2']]
  cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
  num_features = [col for col in all_cols if col not in cat_features]
  return all_cols, cat_features, num_features

def agg_functions(df, num_features, cat_features, numberobs = False#, exclnullCols, 
                  #dummy_nan_col
                  ):
  
  test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'max', 'min', 'last', 'first'])

  print('num agg complete')

  #test_nan_agg = df.groupby("customer_ID")[exclnullCols].agg(['last'])
  ##dummy_nan_col_agg = df.groupby("customer_ID")[dummy_nan_col].agg(['sum'])
  #df_presence_agg = df.groupby("customer_ID")['number_of_observations'].agg(['last'])
  
  test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
  print('cat agg complete')

  df = pd.concat([test_num_agg, test_cat_agg],   #dummy_nan_col_agg, 
                   # test_nan_agg], 
                   axis=1)
  
  print('concat complete')
  df.columns = ['_'.join(x) for x in df.columns]


  for col in num_features:
    if col not in ['number_of_observations']:
      # max = f'{col}_max'
      # min = f'{col}_min'
      # df[f'{col}_minmax'] = df[max] -df[min]
      last = f'{col}_last'
      first = f'{col}_first'
      df[f'{col}_abschange'] = df[last] -df[first]  





  print('drop numberobs')

  if numberobs ==True:
    to_drop = ['number_of_observations_mean',  'number_of_observations_max','number_of_observations_std', 'number_of_observations_min', 'number_of_observations_first']
    df.drop(to_drop, axis = 1, inplace = True)
    df.rename(columns={'number_of_observations_last':'number_of_observations'}, inplace = True)

  print('drop first')
  droplist = list(df.loc[:, df.columns.str.contains('first')].columns)
  df.drop(droplist, axis = 1, inplace = True)


  print('drop numberobs complete')
  del test_num_agg, test_cat_agg
  _ = gc.collect()
  print('shape after engineering', df.shape )
  return df

def add_meandev(df, num_features):
  
  for i in [f for f in num_features if f not in ['number_of_observations']]:
    last = f'{i}_last'
    mean = f'{i}_mean' 
    df[f'{i}_meandev'] = np.nan
    df.loc[(df[last] != np.nan), f'{i}_meandev'] = df[last] -df[mean]

  return df

def dropfirst(df):
  droplist = list(df.loc[:, df.columns.str.contains('first')].columns)
  df.drop(droplist, axis = 1, inplace = True)
  print('shape after engineering', df.shape )
  return df

def add_targets(df, TARGET_PATH):
  # ADD TARGETS
  targets = pd.read_csv(TARGET_PATH)
  targets['customer_ID'] = targets['customer_ID'].str[-16:].apply(int, base =16)
  targets = targets.set_index('customer_ID')
  df = df.merge(targets, left_index=True, right_index=True, how='left', sort = True)
  df.target = df.target.astype('int8')
  del targets

  # NEEDED TO MAKE CV DETERMINISTIC (cudf merge above randomly shuffles rows)
  df = df.reset_index()
  return df


def get_difference(df, num_features, train_set = None, Part =None):
    df1 = []
    customer_ids = []
    for customer_id, cus in tqdm(df.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = cus[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    df1.set_index('customer_ID', inplace = True)
    if train_set == True: df1.to_parquet(f'{SAVE_PATH}diff_{VER}.parquet')
    elif train_set == False: df1.to_parquet(f'{SAVE_PATH}diff_test_{VER}_num{Part}.parquet')
    return df1
  
def onehot_encoding(df, cat_cols):
  cat_cols = [f'{i}_last' for i in cat_cols]
  for col in cat_cols:
    df = pd.get_dummies(df, columns=[col], drop_first=True)
  return df

def get_difference_two(df, num_features, train_set = None, Part =None):
    df2 = []
    customer_ids = []
    for customer_id, cus in tqdm(df.groupby(['customer_ID'])):
        # Get the differences
        diff_df2 = cus[num_features].diff(2).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df2.append(diff_df2)
        customer_ids.append(customer_id)
    # Concatenate
    df2 = np.concatenate(df2, axis = 0)
    # Transform to dataframe
    df2 = pd.DataFrame(df2, columns = [col + '_diff2' for col in df[num_features].columns])
    # Add customer id
    df2['customer_ID'] = customer_ids
    df2.set_index('customer_ID', inplace = True)
    if train_set == True: df2.to_parquet(f'{SAVE_PATH}diff2_{VER}.parquet')
    elif train_set == False: df2.to_parquet(f'{SAVE_PATH}diff2_test_{VER}_num{Part}.parquet')
    return df2


def label_encoding(df, cat_cols):
  cat_cols = [f'{i}_last' for i in cat_cols]
  for col in cat_cols:
    encoder = LabelEncoder()
    df[col] = encoder.fit_transform(df[col])
    return df

def iterative_imputer(df):
  imp = IterativeImputer(
    estimator=xgboost.XGBRegressor(
        #n_estimators=10,
        random_state=SEED,
        tree_method='gpu_hist',
    ),
    missing_values=np.nan,
    #max_iter=20,
    initial_strategy='mean',
    imputation_order='ascending',
    #verbose=100,
    random_state=1
    )

  df[:] = imp.fit_transform(df)
  return df



In [6]:
#all functions compact
def preprocess(PATH = TRAIN_PATH, TARGET_PATH = TARGET_PATH, train_set = True, test = None, Part = None):
  if train_set == True:
    df = read_file(path = TRAIN_PATH)
  else:
    df = test
  print('read file complete')
  df = revertnan(df)
  print('revertnan complete')
  df = numberobs_feature(df)
  print('numberobs complete')
  all_cols, cat_features, num_features = get_features(df)
  print('get features complete')
  #diff1 = get_difference(df, num_features, train_set = train_set, Part = Part)
  if train_set == True: diff1 = pd.read_parquet(f'{SAVE_PATH}diff_12.parquet')
  elif train_set == False: diff1 = pd.read_parquet(f'{SAVE_PATH}diff_test_12_num{Part}.parquet')
  print('get diff complete')
  #diff2 = get_difference_two(df, num_features, train_set = train_set, Part = Part)
  if train_set == True: diff2 = pd.read_parquet(f'{SAVE_PATH}diff2_20.parquet')
  elif train_set == False: diff2 = pd.read_parquet(f'{SAVE_PATH}diff2_test_20_num{Part}.parquet')
  print('get diff2 complete')
  df = agg_functions(df, num_features, cat_features, numberobs = True)
  df = df.merge(diff1, left_index=True, right_index=True, how='left') 
  df = df.merge(diff2, left_index=True, right_index=True, how='left') 
  del diff1
  _ = gc.collect()
  del diff2
  _ = gc.collect()
  print('agg features complete')
  df = add_meandev(df, num_features)
  print('meandev complete')
  df = afterpay(df)
  print('afterpay complete')
  # df = onehot_encoding(df, cat_cols=cat_features)
  # print('onehot complete')
  df = label_encoding(df, cat_cols=cat_features)
  print('label encoding complete')
  # df = add_Bratios(df)
  df = fill_na(df, NAN_VALUE)
  print('fillna complete')
  #df = iterative_imputer(df)
  gc.collect()
  if train_set == True:
    df = add_targets(df, TARGET_PATH)
  return df

In [ ]:
train = preprocess(PATH = TRAIN_PATH)

In [8]:
train.to_parquet(f'{SAVE_PATH}train_{VER}.parquet')

In [7]:
train = pd.read_parquet(f'{SAVE_PATH}train_{VER}.parquet')

In [8]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

In [9]:
data = read_file(path = TRAIN_PATH)

shape of data: (5531451, 190)


In [10]:
all_cols, cat_features, num_features = get_features(data)

In [11]:
del data

In [12]:
cat_features = [f"{cf}_last" for cf in cat_features]

In [13]:
gc.collect()

151

In [14]:
#del train, test
gc.collect()
#train['number_of_observations']

22

In [17]:
def get_feature_list(df):
  features = df.columns[1:-1]
  print(f'There are {len(features)} features!')
  return features

In [18]:
features = get_feature_list(train)

There are 1643 features!


In [ ]:
features

In [17]:
def save_features(features):
  with open(f'{SAVE_PATH}features_V{VER}.csv', 'w') as csvfile:
    # creating a csv writer object
    writer = csv.writer(csvfile)    
    writer.writerow(features) 

In [18]:
save_features(features)

In [ ]:
del features

In [15]:
features = pd.read_csv(f'{SAVE_PATH}features_V{VER}.csv')
features = pd.Index(features.columns)


Train XGB

In [16]:
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [18]:
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric_mod(y_true, y_pred), True

In [17]:
def get_lgb_parameters():
  
  lgb_parms = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'dart',
        'seed': SEED,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 2,
        'min_data_in_leaf': 40,
        'device_type': 'gpu'
        }
  
  
  
  
  return lgb_parms

In [19]:
lgb_parms = get_lgb_parameters()

In [20]:
def train_model(df, SEED=SEED, SAVE_PATH =SAVE_PATH, VER=VER, cat_features = cat_features):
  features_importance= pd.DataFrame({'Feature':[], 'Importance':[]})
  oof = []
  TRAIN_SUBSAMPLE = 1.0
  gc.collect()
  kaggle_metrics_folds =[]

  skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
  for fold,(train_idx, valid_idx) in enumerate(skf.split(
            df, df.target )):

    
    print('#'*25)
    print('### Fold',fold+1)
    print('### Train size',len(train_idx),'Valid size',len(valid_idx))
    print(f'### Training with {int(TRAIN_SUBSAMPLE*100)}% fold data...')
    print('#'*25)
    
    # TRAIN, VALID, TEST FOR FOLD K
    X_train = df.loc[train_idx, features]
    y_train = df.loc[train_idx, 'target']

    X_valid = df.loc[valid_idx, features]
    y_valid = df.loc[valid_idx, 'target']
    

    lgb_train = lgb.Dataset(X_train, y_train, categorical_feature = cat_features)
    lgb_valid = lgb.Dataset(X_valid, y_valid, categorical_feature = cat_features)

    print('datasets created')
    
    # TRAIN MODEL FOLD K

    model = lgb.train(
            params = lgb_parms,
            train_set = lgb_train,
            num_boost_round = 10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 1500,
            verbose_eval = 300
            #feval = lgb_amex_metric
            )
    model.save_model(f'{SAVE_PATH}XGB_v{VER}_fold{fold}.xgb')
    
    # # GET FEATURE IMPORTANCE FOR FOLD K
    # dd = model.get_score(importance_type='weight')
    # df_pred = pd.DataFrame({'feature':dd.keys(),f'importance_{fold}':dd.values()})
    # importances.append(df_pred)

    fold_importance_df= pd.DataFrame({'Feature':[], 'Importance':[]})
    fold_importance_df['Feature']= features
    fold_importance_df['Importance']= model.feature_importance()
    fold_importance_df["fold"] = fold + 1
    features_importance = pd.concat([features_importance, fold_importance_df], axis=0)
            
    # INFER OOF FOLD K
    oof_preds = model.predict(df.loc[valid_idx, features], num_iteration=model.best_iteration)
    acc = amex_metric_mod(y_valid.values, oof_preds)
    print('Kaggle Metric =',acc,'\n')
    kaggle_metrics_folds.append(acc)
    
    # SAVE OOF
    df_pred = df.loc[valid_idx, ['customer_ID','target'] ].copy()
    df_pred['oof_pred'] = oof_preds
    oof.append( df_pred )
    
    del  X_train, y_train #,Xy_train,
    del X_valid, y_valid, model
    _ = gc.collect()
    
  print('#'*25)
  oof = pd.concat(oof,axis=0,ignore_index=True).set_index('customer_ID')
  acc = amex_metric_mod(oof.target.values, oof.oof_pred.values)
  print('OVERALL CV Kaggle Metric =',acc)
  print(kaggle_metrics_folds)


  # imp = importances[0].copy()
  # for k in range(1,FOLDS): imp = imp.merge(importances[k], on='feature', how='left')
  # imp['importance'] = imp.iloc[:,1:].mean(axis=1)
  # imp = imp.sort_values('importance',ascending=False)
  # imp.to_csv(f'xgb_feature_importance_v{VER}.csv',index=False)
  return oof, features_importance

In [21]:
oof, features_importance = train_model(train)

#########################
### Fold 1
### Train size 367130 Valid size 91783
### Training with 100% fold data...
#########################
datasets created
[LightGBM] [Info] Number of positive: 95062, number of negative: 272068
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 266820
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 1637
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 808 dense feature groups (282.90 MB) transferred to GPU in 0.205022 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051523
[LightGBM] [Info] Start training from score -1.051523
[300]	training's binary_logloss: 0.391992	valid_1's binary_logloss: 0.394399
[600]	training's binary_loglos

In [22]:
oof.to_parquet(f'{SAVE_PATH}oof_V{VER}.csv')
features_importance.to_parquet(f'{SAVE_PATH}importance_V{VER}.csv')

In [ ]:
def get_header(df):
  header=['Version']
  for col in df.columns:
    if col not in ['customer_ID', 'S_2']:
      cname = f'{col}_lower'
      header.append(cname)
      cname = f'{col}_upper'
      header.append(cname)
  return header


header = get_header(train_agg)

with open(f'{SAVE_PATH}oof_results.csv', 'w') as csvfile: 
    # creating a csv writer object 
    writer = csv.writer(csvfile) 

    #header = ['Version'] +list(train_agg.columns)    
    writer.writerow(header) 

In [30]:
gc.collect()

63

In [ ]:
breakdown

In [34]:
breakdown.to_parquet(f'{SAVE_PATH}breakdown_V{VER}.csv')

In [42]:
oof

,target,oof_pred
customer_ID,,
195745370954238,0,0.000196
861377627745855,0,0.000675
928884405244202,0,0.044520
1239928347876539,0,0.003082
1243572453994939,0,0.007462
...,...,...
18445652994177242937,0,0.005339
18445805218934687167,0,0.001437
18446212425763655834,0,0.024474




```
base = 0.791536
ohne na Behandlung = 0.790870
na fill nach agg = 0.7916640
revertnan und na_fill nach agg = 0.791823
mit number of obs (und na_fill nach agg) = 0.79223834
so mit growth feature = 0.79185923 ??
mit abschange = 0.7917617 ??
mit abschange nicht skaliert = 0.791859 ?
number of obs in buckets (größer 10) + abschange = 0.7915386 ?
mit number_of_obs und min-max = 0.792181
mit number of obs (und na_fill nach agg)fillna vor meandev + mean deviation = 0.792464
mit number of obs (und na_fill nach agg) fillna nach meandev + mean deviation = 0.79237036 (=VER 1, 0.794 public lb)
s.o. + after pay features = 0.79310
VER 3: s.o. mit getunten Hyperparametern = 0.794355 (V3, 0.796 public lb)
s.o. mit B_29 = 0.794610 (v5, 0.796 public lb)
so ohne B_29 mit 1:100 Bratios = 0.7951046
so mit allen Bratios = 0.7952144 (0.795 lb)
VER 3 + first: 0.794577
Ver 3 + forst + first diff = 0.7955055
Ver 3 +  first diff = 0.79504522 ( 0.796735284564001, 0.79686627331464, 0.7968411263507417, 0.7920236043263609, 0.793339220870202 )

V7 = V3 + first + first diff = OVERALL CV Kaggle Metric = 0.7951661498772574 (LB 0.797)
[0.7962198773071689, 0.7965380847686576, 0.7972899788446302, 0.7909934597488399, 0.794277615381761] (Unterschied zu so 0.7955 konnte nicht nachvollzogen werden)

V9 = V9 mit getunten hyperparametern :OVERALL CV Kaggle Metric = 0.795368601110926
[0.7954320326411877, 0.7974078889361778, 0.7973913538116686, 0.7913097036060067, 0.7957431778045534]

V12: after pay nur mit last, one hot encoding OVERALL CV Kaggle Metric = 0.7957373107518337 (lb 0.797)
[0.7969486198782119, 0.7962363538077044, 0.7973269062536987, 0.7920697622584665, 0.7955655878459955]

V13: mit maxmin, ohne std, ohne min, OVERALL CV Kaggle Metric = 0.7953608849564417 (0.796 lb)
[0.7968882798164574, 0.7964878206575973, 0.7974681877663161, 0.7917797910863018, 0.793948382819542]

V14: mit min, ohne std: OVERALL CV Kaggle Metric = 0.7952813107927237
[0.7969120719048222, 0.7965001952586592, 0.7967465309957907, 0.7922152777481679, 0.7941480645671318]

V15: so mit abs change: 0.7952517408213524
[0.7973194651977258, 0.7966489899997335, 0.7973107084598802, 0.7926435788160571, 0.7921604620461141]

V16: wie 15 mit ordinal encoder OVERALL CV Kaggle Metric = 0.794532772359821
[0.7972029207650211, 0.7965792540555843, 0.7963623626951393, 0.7914712548912595, 0.7919712944759572]

V17: wie 12 mit stratifiedkfold: OVERALL CV Kaggle Metric = 0.7955119445098058
[0.7940790990360199, 0.7933681806036881, 0.7991199768804013, 0.7928613145718828, 0.7980624519463699] (lb 0.796)

V18 lgb OVERALL CV Kaggle Metric = 0.7967385072608157
[0.7951264519106611, 0.7954861934823085, 0.8002551252292345, 0.7920291716075025, 0.8000457888549861]

V16 (kapo) mit diff2 = 0.7959986936294856
[0.7974611116231001, 0.7962414905558102, 0.7978782783323859, 0.793126064579345, 0.7948356461615571] (lb 0.796)


Prepare Testdata

In [26]:
# CALCULATE SIZE OF EACH SEPARATE TEST PART
def get_rows(customers, test, NUM_PARTS = NUM_PARTS, verbose = ''):
    chunk = len(customers)//NUM_PARTS
    if verbose != '':
        print(f'We will process {verbose} data as {NUM_PARTS} separate parts.')
        print(f'There will be {chunk} customers in each part (except the last part).')
        print('Below are number of rows in each part:')
    rows = []

    for k in range(NUM_PARTS):
        if k==NUM_PARTS-1: cc = customers[k*chunk:]
        else: cc = customers[k*chunk:(k+1)*chunk]
        s = test.loc[test.customer_ID.isin(cc)].shape[0]
        rows.append(s)
    if verbose != '': print( rows )
    return rows,chunk

In [27]:
def get_rowsnumcust(TEST_PATH, NUM_PARTS=NUM_PARTS):
  print(f'Reading test data...')
  test = read_file(path = TEST_PATH, usecols = ['customer_ID','S_2'])
  customers = test[['customer_ID']].drop_duplicates().sort_index().values.flatten()
  rows,num_cust = get_rows(customers, test[['customer_ID']], NUM_PARTS = NUM_PARTS, verbose = 'test')
  return rows,num_cust,customers

In [28]:
rows,num_cust, customers = get_rowsnumcust(TEST_PATH, NUM_PARTS=NUM_PARTS)

Reading test data...
shape of data: (11363762, 2)
We will process test data as 10 separate parts.
There will be 92462 customers in each part (except the last part).
Below are number of rows in each part:
[1136415, 1137255, 1135580, 1135734, 1136082, 1137166, 1136612, 1137228, 1136301, 1135389]


In [29]:
# INFER TEST DATA IN PARTS
skip_rows = 0
skip_cust = 0
test_preds = []

for k in range(NUM_PARTS):
  # READ PART OF TEST DATA
  print(f'\nReading test data...')
  test = read_file(path = TEST_PATH)
  test = test.sort_index()
  test = test.iloc[skip_rows:skip_rows+rows[k]]
  skip_rows += rows[k]
  print(f'=> Test part {k+1} has shape', test.shape )
    
  # PROCESS AND FEATURE ENGINEER PART OF TEST DATA
  test = preprocess(train_set = False, test = test, Part =k)
  if k==NUM_PARTS-1: test = test.loc[customers[skip_cust:]]
  else: test = test.loc[customers[skip_cust:skip_cust+num_cust]]
  skip_cust += num_cust
 
   
  #ammend for one hot encoding
  test['D_64_last_1.0'] = 0
  test['D_66_last_1.0'] = 0
  test['D_68_last_1.0'] = 0
  # TEST DATA FOR XGB
  X_test = test[features]
  print('X_test complete')
  dtest = xgb.DMatrix(data=X_test)
  print('dtest complete')
  #test = test[['P_2_mean']] # reduce memory
  del test, X_test
  gc.collect()
  gc.collect()

  # INFER XGB MODELS ON TEST DATA
  model = xgb.Booster()
  model.load_model(f'{SAVE_PATH}XGB_v{VER}_fold0.xgb')
  print('load model complete')
  preds = model.predict(dtest)
  print('preds complete')
  for f in range(1,FOLDS):
    del model
    gc.collect()
    model = xgb.Booster()
    model.load_model(f'{SAVE_PATH}XGB_v{VER}_fold{f}.xgb')
    print(f'load {f} complete')
    preds += model.predict(dtest)
    print(f'preds {f} complete')
  preds /= FOLDS
  test_preds.append(preds)

  # CLEAN MEMORY
  del dtest, model
  _ = gc.collect()

test_preds = np.concatenate(test_preds)
#test = cudf.DataFrame(index=customers,data={'prediction':test_preds})
test = pd.DataFrame(index=customers,data={'prediction':test_preds})
#sub = cudf.read_csv(SUBMISSION_FILE_PATH)[['customer_ID']]
sub = pd.read_csv(SUBMISSION_FILE_PATH)[['customer_ID']]
#sub['customer_ID_hash'] = sub['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
sub['customer_ID_hash'] = sub['customer_ID'].str[-16:].apply(int, base =16)
sub = sub.set_index('customer_ID_hash')
#sub = sub.set_index('customer_ID')
sub = sub.merge(test[['prediction']], left_index=True, right_index=True, how='left')
sub = sub.reset_index(drop=True)

sub.to_csv(f'{SAVE_PATH}submission_xgb_v{VER}.csv',index=False)
print('Submission file shape is', sub.shape )
sub.head()


Reading test data...
shape of data: (11363762, 190)
=> Test part 1 has shape (1136415, 190)
read file complete
revertnan complete
numberobs complete
get features complete
get diff complete
num agg complete
cat agg complete
concat complete
drop numberobs
drop numberobs complete
shape after engineering (92462, 1096)
agg features complete


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


meandev complete
afterpay complete


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


onehot complete
fillna complete
X_test complete
dtest complete
load model complete
preds complete
load 1 complete
preds 1 complete
load 2 complete
preds 2 complete
load 3 complete
preds 3 complete
load 4 complete
preds 4 complete

Reading test data...
shape of data: (11363762, 190)
=> Test part 2 has shape (1137255, 190)
read file complete
revertnan complete
numberobs complete
get features complete
get diff complete
num agg complete
cat agg complete
concat complete
drop numberobs
drop numberobs complete
shape after engineering (92462, 1096)
agg features complete
meandev complete
afterpay complete
onehot complete
fillna complete
X_test complete
dtest complete
load model complete
preds complete
load 1 complete
preds 1 complete
load 2 complete
preds 2 complete
load 3 complete
preds 3 complete
load 4 complete
preds 4 complete

Reading test data...
shape of data: (11363762, 190)
=> Test part 3 has shape (1135580, 190)
read file complete
revertnan complete
numberobs complete
get features com

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.038103
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000712
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.028836
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.178760
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.879854


In [43]:
gc.collect()

631

### ENDE

In [ ]:
train


,customer_ID,P_2_mean,P_2_std,P_2_max,P_2_min,P_2_last,P_2_first,D_39_mean,D_39_std,D_39_max,...,B_17-P_3_meandev,D_39-P_2_meandev,D_39-P_3_meandev,D_131-P_2_meandev,D_131-P_3_meandev,S_16-P_2_meandev,S_16-P_3_meandev,S_23-P_2_meandev,S_23-P_3_meandev,target
0,3249127622875,1.003573,0.001341,1.005004,1.002345,1.005004,1.002345,2.846154,6.841615,21,...,-127.000000,-0.001431,9.662469,-0.000816,-0.004198,-0.002197,-0.006118,-0.003054,-0.002010,0
1,23402014749356,0.415518,0.100825,0.498842,0.188222,0.215565,0.463519,3.230769,4.657500,15,...,0.398372,-2.030815,-1.767706,0.952465,1.215575,0.518049,0.781159,0.201210,0.464319,1
2,37423844044824,0.621856,0.032564,0.689850,0.583851,0.583851,0.689850,12.000000,12.006942,46,...,0.151129,1.038004,0.975343,0.038005,-0.024657,0.041275,-0.021386,0.418291,0.355629,1
3,149683802139734,0.913329,0.034975,0.976633,0.845514,0.931456,0.976633,3.230769,7.886210,21,...,-127.000000,-3.248897,-3.290519,-0.018127,-0.059750,-0.015045,-0.056667,0.048592,0.006970,0
4,160820792638518,0.555897,0.046583,0.662159,0.468876,0.518131,0.468876,1.384615,3.477031,11,...,-0.018422,-1.346849,-1.279861,0.037766,0.104755,0.040179,0.107168,0.040386,0.107375,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,18446623475205808948,0.342072,0.103027,0.486856,0.115548,0.115548,0.486856,2.076923,3.593085,11,...,0.273784,-1.850400,-1.783496,0.226523,0.293427,-0.861105,-0.794201,0.301186,0.368090,1
458909,18446629179309953275,0.242426,0.054706,0.317476,0.199567,0.215042,0.317476,13.571429,10.373960,24,...,0.000515,8.194051,5.000153,0.029562,0.002935,0.031139,0.003970,0.030783,0.002520,0
458910,18446635398005430687,0.541245,0.097363,0.666280,0.310770,0.445689,0.666280,0.000000,0.000000,0,...,-0.218169,0.095556,0.098894,0.095556,0.098894,5.205449,5.142211,0.002523,-0.002024,0
458911,18446654790786387807,0.996442,0.013313,1.009017,0.974290,0.974290,1.004322,0.000000,0.000000,0,...,-127.000000,0.022152,0.029393,0.022152,0.029393,0.022832,0.030073,0.024370,0.031610,0


In [8]:
#train test split
train_df, test_df = train_test_split(train, test_size=0.25, stratify=train['target'])

In [ ]:
!pip install optuna
import optuna

In [10]:
# optuna

def objective(trial,  train_df, test_df, features):
    
    param = {
        'booster':'gbtree',
        'tree_method':'gpu_hist', 
        "objective": "binary:logistic",
        'lambda': trial.suggest_loguniform(
            'lambda', 1e-3, 10.0
        ),
        'alpha': trial.suggest_loguniform(
            'alpha', 1e-3, 10.0
        ),
        'colsample_bytree': trial.suggest_float(
            'colsample_bytree', 0.5,1,step=0.1
        ),
        'subsample': trial.suggest_float(
            'subsample', 0.5,1,step=0.1
        ),
        'learning_rate': trial.suggest_float(
            'learning_rate', 0.01,0.05,step=0.005
        ),
        'max_depth': trial.suggest_int(
            'max_depth', 4,9,1
        ),
        'random_state': 99,
        'eval_metric':'logloss',
        'min_child_weight': trial.suggest_int(
            'min_child_weight', 1,256,1
        ),
        
    }
    
    

    
    
    
    
    oof = []
    gc.collect()
    

        
    # TRAIN, VALID, TEST FOR FOLD K
    #Xy_train = IterLoadForDMatrix(train_df, features, 'target')

    X_train = train_df.loc[:, features]
    y_train = train_df.loc[:, 'target']    
        
    X_valid = test_df.loc[:, features]
    y_valid = test_df.loc[:, 'target']
      
    #dtrain = xgb.DeviceQuantileDMatrix(Xy_train, max_bin=256)

    dtrain = xgb.DMatrix(data=X_train, label=y_train)
    dvalid = xgb.DMatrix(data=X_valid, label=y_valid)
      
        

        
        # TRAIN MODEL FOLD K
    model = xgb.train(param, 
                  dtrain=dtrain,
                  evals=[(dtrain,'train'),(dvalid,'valid')],
                  num_boost_round=9999,
                  early_stopping_rounds=100,
                  verbose_eval=300
                  ) 
      
        
        # INFER OOF FOLD K
    oof_preds = model.predict(dvalid)
        #acc = amex_metric_mod(y_valid.values, oof_preds)
    
    
        # SAVE OOF
    df = test_df.loc[:, ['customer_ID','target'] ].copy()
    df['oof_pred'] = oof_preds
    oof.append(df)

              


    del dtrain,  df
    del X_valid, y_valid, dvalid, model
    _ = gc.collect()
    

    oof = pd.concat(oof,axis=0,ignore_index=True).set_index('customer_ID')
    metric = amex_metric_mod(oof.target.values, oof.oof_pred.values)
        
    return metric

In [14]:
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(
        trial,  train_df, test_df, features), n_trials= 100) 

[I 2022-08-13 21:53:22,205] A new study created in memory with name: no-name-32a45eb6-bed4-492d-840b-203cc1dfa9cc


[0]	train-logloss:0.686615	valid-logloss:0.686655
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[300]	train-logloss:0.243862	valid-logloss:0.250032
[600]	train-logloss:0.219835	valid-logloss:0.22928
[900]	train-logloss:0.212013	valid-logloss:0.224505
[1200]	train-logloss:0.207185	valid-logloss:0.222199
[1500]	train-logloss:0.203544	valid-logloss:0.220863
[1800]	train-logloss:0.200586	valid-logloss:0.220025
[2100]	train-logloss:0.197999	valid-logloss:0.219435
[2400]	train-logloss:0.195573	valid-logloss:0.218989
[2700]	train-logloss:0.193307	valid-logloss:0.218655
[3000]	train-logloss:0.191194	valid-logloss:0.218389
[3300]	train-logloss:0.189116	valid-logloss:0.218176
[3600]	train-logloss:0.187047	valid-logloss:0.217978
[3900]	train-logloss:0.185112	valid-logloss:0.217818
[4200]	train-logloss:0.18318	valid-logloss:0.217687
[4500]	train-logloss:0.181388	valid-logloss:0.217586
[4800]	t

[I 2022-08-13 22:05:59,232] Trial 0 finished with value: 0.7941231955570944 and parameters: {'lambda': 6.462217156923637, 'alpha': 4.514849216110904, 'colsample_bytree': 0.6, 'subsample': 1.0, 'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 94}. Best is trial 0 with value: 0.7941231955570944.


[0]	train-logloss:0.674262	valid-logloss:0.674346
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[300]	train-logloss:0.223038	valid-logloss:0.229291
[600]	train-logloss:0.213376	valid-logloss:0.222885
[900]	train-logloss:0.208538	valid-logloss:0.220829
[1200]	train-logloss:0.204942	valid-logloss:0.219831
[1500]	train-logloss:0.201735	valid-logloss:0.21915
[1800]	train-logloss:0.198756	valid-logloss:0.218729
[2100]	train-logloss:0.196011	valid-logloss:0.21839
[2400]	train-logloss:0.193434	valid-logloss:0.218195
[2700]	train-logloss:0.191046	valid-logloss:0.218062
[3000]	train-logloss:0.1887	valid-logloss:0.218022
[3300]	train-logloss:0.18633	valid-logloss:0.217943
Stopping. Best iteration:
[3368]	train-logloss:0.185856	valid-logloss:0.217933



[I 2022-08-13 22:11:48,614] Trial 1 finished with value: 0.7916748342573557 and parameters: {'lambda': 0.023418138366062092, 'alpha': 4.4303464238534875, 'colsample_bytree': 0.8, 'subsample': 1.0, 'learning_rate': 0.03, 'max_depth': 4, 'min_child_weight': 121}. Best is trial 0 with value: 0.7941231955570944.


...

[I 2022-08-14 09:43:36,860] Trial 97 finished with value: 0.794794299794604 and parameters: {'lambda': 0.15317280046978893, 'alpha': 0.8934256326428722, 'colsample_bytree': 0.6, 'subsample': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'min_child_weight': 66}. Best is trial 58 with value: 0.7953711156103191.


[0]	train-logloss:0.686498	valid-logloss:0.686549
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[300]	train-logloss:0.238503	valid-logloss:0.246758
[600]	train-logloss:0.214384	valid-logloss:0.227078
[900]	train-logloss:0.205832	valid-logloss:0.222683
[1200]	train-logloss:0.200004	valid-logloss:0.220668
[1500]	train-logloss:0.195349	valid-logloss:0.219553
[1800]	train-logloss:0.191141	valid-logloss:0.218792
[2100]	train-logloss:0.18736	valid-logloss:0.218304
[2400]	train-logloss:0.183776	valid-logloss:0.217989
[2700]	train-logloss:0.180547	valid-logloss:0.217742
[3000]	train-logloss:0.17733	valid-logloss:0.217548
[3300]	train-logloss:0.174253	valid-logloss:0.217413
[3600]	train-logloss:0.171372	valid-logloss:0.21732
[3900]	train-logloss:0.168559	valid-logloss:0.217214
[4200]	train-logloss:0.165726	valid-logloss:0.217144
Stopping. Best iteration:
[4354]	train-logloss:0.164286	valid-

[I 2022-08-14 09:53:43,629] Trial 98 finished with value: 0.7946222226057015 and parameters: {'lambda': 5.454102084676547, 'alpha': 0.5950808157878608, 'colsample_bytree': 0.6, 'subsample': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'min_child_weight': 67}. Best is trial 58 with value: 0.7953711156103191.


[0]	train-logloss:0.686483	valid-logloss:0.686532
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[300]	train-logloss:0.238286	valid-logloss:0.246507
[600]	train-logloss:0.214279	valid-logloss:0.226914
[900]	train-logloss:0.205947	valid-logloss:0.222599
[1200]	train-logloss:0.200272	valid-logloss:0.220639
[1500]	train-logloss:0.195824	valid-logloss:0.219512
[1800]	train-logloss:0.191776	valid-logloss:0.218783
[2100]	train-logloss:0.188162	valid-logloss:0.218314
[2400]	train-logloss:0.184721	valid-logloss:0.21798
[2700]	train-logloss:0.181585	valid-logloss:0.217765
[3000]	train-logloss:0.178603	valid-logloss:0.21756
[3300]	train-logloss:0.175682	valid-logloss:0.217407
[3600]	train-logloss:0.172933	valid-logloss:0.217291
[3900]	train-logloss:0.170271	valid-logloss:0.2172
[4200]	train-logloss:0.167559	valid-logloss:0.217148
Stopping. Best iteration:
[4156]	train-logloss:0.167925	valid-l

[I 2022-08-14 10:03:24,483] Trial 99 finished with value: 0.7941889560916399 and parameters: {'lambda': 0.34337791530443534, 'alpha': 0.9978144985391355, 'colsample_bytree': 0.6, 'subsample': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'min_child_weight': 77}. Best is trial 58 with value: 0.7953711156103191.
